In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

2.1. What is the median age of the training set? You can easily modify the age in the
dataframe. Note that you need to modify the code above a bit to fill with mode() because mode() returns
a series rather than a single value.

In [10]:
df["Age"] = df["Age"].fillna(df["Age"].median())

In [11]:
df["Age"].isnull().sum()

0

2.2. Some fields like ‘Embarked’ are categorical. They need to be converted to numbers
first. We will represent S with 0, C with 1, and Q with 2. What is the mode of Embarked?
Fill the missing values with the mode. You can set the value of Embarked easily with the
following command. Do the same for Sex.

In [12]:
df.loc[:,"Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])

In [13]:
df["Embarked"].isnull().sum()

0

In [14]:
df.loc[df["Embarked"] == "S", "Embarked"] = 0
df.loc[df["Embarked"] == "C", "Embarked"] = 1
df.loc[df["Embarked"] == "Q", "Embarked"] = 2

In [15]:
df.loc[df["Sex"] == "male", "Sex"] = 0
df.loc[df["Sex"] == "female", "Sex"] = 1

In [16]:
df["Embarked"]

0      0
1      1
2      0
3      0
4      0
      ..
886    0
887    0
888    0
889    1
890    2
Name: Embarked, Length: 891, dtype: object

2.3. Write a logistic regression classifier using gradient descent (from scratch, don’t
use scikit-learn framework or others) as learned in class (Survived). Use PClass, Sex, Age,
and Embarked as input features.

In [17]:
import numpy as np

class LogisticRegressionSelfMade:
    def __init__(self, lr=0.01, iterations=1000, threshold=0.5):
        self.lr = lr
        self.iterations = iterations
        self.threshold = threshold
        self.weights = None
        self.bias = None

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # 1. Initial weights
        self.weights = np.zeros(n_features)
        self.bias = 0

        # 2. Gradient Descent Loop
        for _ in range(self.iterations):
            # Linear model
            linear_model = np.dot(X, self.weights) + self.bias
            # Send model into Sigmoid
            y_predicted = self._sigmoid(linear_model)

            # 3. calculate gredient
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            # 4. update parameter
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)

        # If predicted value more than threshold -> 1 else -> 0
        return [1 if i > self.threshold else 0 for i in y_predicted]

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split Data into Train:Test = 80:20
train_data_23, test_data_23 = train_test_split(df, test_size=0.2, random_state=42)
X_train_23 = np.array(train_data_23[["Pclass","Sex","Age","Embarked"]].values, dtype = float)
X_test_23 = np.array(test_data_23[["Pclass","Sex","Age","Embarked"]].values, dtype = float)

# Normalize Data
scaler = StandardScaler()
X_train_23 = scaler.fit_transform(X_train_23)
y_train_23 = np.array(train_data_23[["Survived"]].values, dtype = float)

X_test_23 = scaler.transform(X_test_23)
y_test_23 = np.array(test_data_23[["Survived"]].values, dtype = float)

In [19]:
X_train_23 = np.array(X_train_23)
y_train_23 = np.array(y_train_23).ravel()

# Call model class
model_23 = LogisticRegressionSelfMade()
model_23.fit(X_train_23, y_train_23)

# Predict
y_train_pred_23 = model_23.predict(np.array(X_train_23))
y_test_pred_23 = model_23.predict(np.array(X_test_23))

# Show Parameter
print(f"Weights: {model_23.weights}")
print(f"Bias: {model_23.bias}")

Weights: [-0.62039548  0.98886625 -0.18008002  0.14993779]
Bias: -0.5109011213832341


2.4 When you evaluate the trained model on the test set, you will need to
make a final decision. Since logistic regression outputs a score between 0 and 1,
you will need to decide whether a score of 0.3 (or any other number) means the
passenger survive or not. For now, we will say if the score is greater than or equal
to 0.5, the passenger survives. If the score is lower than 0.5 the passenger will be
dead. This process is often called ‘Thresholding.’ Then, show your precision, recall,
f1-score on the training set and test set.

In [20]:
from sklearn.metrics import classification_report

print(classification_report(y_train_23, y_train_pred_23))
print(classification_report(y_test_23, y_test_pred_23))


              precision    recall  f1-score   support

         0.0       0.82      0.85      0.83       444
         1.0       0.73      0.68      0.71       268

    accuracy                           0.79       712
   macro avg       0.77      0.77      0.77       712
weighted avg       0.78      0.79      0.78       712

              precision    recall  f1-score   support

         0.0       0.81      0.83      0.82       105
         1.0       0.75      0.72      0.73        74

    accuracy                           0.78       179
   macro avg       0.78      0.77      0.77       179
weighted avg       0.78      0.78      0.78       179



2.5 Try adding some higher order features to your training e.g. (x1^2,
x1,x2,...) and explain why these non-linear features are introduced (you can use
concepts in the ML class). Does this model have better accuracy on the training
set? How does it perform on the test set?

In [21]:
df_add_feature = df.assign(Age_square = df['Age'] ** 2)

In [22]:
# Split Data into Train:Test = 80:20
train_data_25, test_data_25 = train_test_split(df_add_feature, test_size=0.2, random_state=42)

# X_train and X_test with Age_square Feature
X_train_25 = np.array(train_data_25[["Pclass","Sex","Age","Embarked","Age_square"]].values, dtype = float)
X_test_25 = np.array(test_data_25[["Pclass","Sex","Age","Embarked","Age_square"]].values, dtype = float)

# Normalize Data
scaler = StandardScaler()
X_train_25 = scaler.fit_transform(X_train_25)
y_train_25 = np.array(train_data_25[["Survived"]].values, dtype = float)

X_test_25 = scaler.transform(X_test_25)
y_test_25 = np.array(test_data_25[["Survived"]].values, dtype = float)

In [23]:
X_train_25 = np.array(X_train_25)
y_train_25 = np.array(y_train_25).ravel()
model_25 = LogisticRegressionSelfMade()
model_25.fit(X_train_25, y_train_25)

y_train_pred_25 = model_25.predict(np.array(X_train_25))
y_test_pred_25 = model_25.predict(np.array(X_test_25))

print(f"Weights: {model_25.weights}")
print(f"Bias: {model_25.bias}")

Weights: [-0.62410393  0.98825074 -0.16149115  0.14938313 -0.02596238]
Bias: -0.5109680039347027


In [24]:
print(classification_report(y_train_25, y_train_pred_25))
print(classification_report(y_test_25, y_test_pred_25))

              precision    recall  f1-score   support

         0.0       0.82      0.85      0.83       444
         1.0       0.73      0.69      0.71       268

    accuracy                           0.79       712
   macro avg       0.78      0.77      0.77       712
weighted avg       0.79      0.79      0.79       712

              precision    recall  f1-score   support

         0.0       0.81      0.84      0.82       105
         1.0       0.76      0.72      0.74        74

    accuracy                           0.79       179
   macro avg       0.78      0.78      0.78       179
weighted avg       0.79      0.79      0.79       179



2.6 What happens if you reduce the number of features to just Sex and Age?

In [25]:
train_data_26, test_data_26 = train_test_split(df, test_size=0.2, random_state=42)
X_train_26 = np.array(train_data_26[["Sex","Age"]].values, dtype = float)
X_test_26 = np.array(test_data_26[["Sex","Age"]].values, dtype = float)

scaler = StandardScaler()
X_train_26 = scaler.fit_transform(X_train_26)
y_train_26 = np.array(train_data_26[["Survived"]].values, dtype = float)

X_test_26 = scaler.transform(X_test_26)
y_test_26 = np.array(test_data_26[["Survived"]].values, dtype = float)

In [26]:
X_train_26 = np.array(X_train_26)
y_train_26 = np.array(y_train_26).ravel()
model_26 = LogisticRegressionSelfMade()
model_26.fit(X_train_26, y_train_26)

y_train_pred_26 = model_26.predict(np.array(X_train_26))
y_test_pred_26 = model_26.predict(np.array(X_test_26))

print(f"Weights: {model_26.weights}")
print(f"Bias: {model_26.bias}")

Weights: [ 1.01811525 -0.03636307]
Bias: -0.4959826038914322


In [27]:
print(classification_report(y_train_26, y_train_pred_26))
print(classification_report(y_test_26, y_test_pred_26))

              precision    recall  f1-score   support

         0.0       0.81      0.86      0.83       444
         1.0       0.74      0.68      0.71       268

    accuracy                           0.79       712
   macro avg       0.78      0.77      0.77       712
weighted avg       0.79      0.79      0.79       712

              precision    recall  f1-score   support

         0.0       0.80      0.84      0.82       105
         1.0       0.75      0.70      0.73        74

    accuracy                           0.78       179
   macro avg       0.78      0.77      0.77       179
weighted avg       0.78      0.78      0.78       179



2.7 We want to show that matrix inversion yields the same answer as the
gradient descent method. However, there is no closed form solution for logistic
regression. Thus, we will use normal linear regression instead. Re-do the Titanic
task as a regression problem by using linear regression. Use the gradient descent
method. Then, show your weights and loss values of training set and test set
(Mean Squared Errors (MSE)).

In [28]:
import numpy as np

class LinearRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.lr = learning_rate
        self.n_iters = n_iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # 1. Initial weights
        self.weights = np.zeros(n_features)
        self.bias = 0

        # 2. Gradient Descent Loop
        for _ in range(self.n_iters):
            y_predicted = np.dot(X, self.weights) + self.bias
            
            # 3. Calculate Gradient
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            # 4. Update Parameter
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias


In [29]:
# Split data into Train:Test = 80:20
train_data_27, test_data_27 = train_test_split(df, test_size=0.2, random_state=42)
X_train_27 = np.array(train_data_27[["Pclass","Sex","Age","Embarked"]].values, dtype = float)
X_test_27 = np.array(test_data_27[["Pclass","Sex","Age","Embarked"]].values, dtype = float)

# Normalize Data
scaler = StandardScaler()
X_train_27 = scaler.fit_transform(X_train_27)
y_train_27 = np.array(train_data_27[["Survived"]].values, dtype = float)

X_test_27 = scaler.transform(X_test_27)
y_test_27 = np.array(test_data_27[["Survived"]].values, dtype = float)

In [30]:
X_train_27 = np.array(X_train_27)
y_train_27 = np.array(y_train_27).ravel()

model_27 = LinearRegression()
model_27.fit(X_train_27, y_train_27)

y_train_pred_27 = model_27.predict(np.array(X_train_27))
y_test_pred_27 = model_27.predict(np.array(X_test_27))

print(f"Weights: {model_27.weights}")
print(f"Bias: {model_27.bias}")

Weights: [-0.143752    0.23739856 -0.05092529  0.0304873 ]
Bias: 0.37638824453046915


In [31]:
train_mse_27 = np.mean((y_train_27 - y_train_pred_27)**2)
test_mse_27 = np.mean((y_test_27 - y_test_pred_27)**2)
print(f"Train MSE: {train_mse_27:.4f}")
print(f"Test MSE: {test_mse_27:.4f}")

Train MSE: 0.1475
Test MSE: 0.3383


2.8. Now try using matrix inversion instead. However, are the weights
learned from the two methods similar? Report the Mean Squared Errors (MSE)
of the difference between the two weights (2.7 and 2.8).

$$\theta = (X^T X)^{-1} X^T y$$

In [32]:
def calculate_theta(X, y):
    """
    X: Features
    y: Target
    """
    # 1. Calculate X Transpose
    X_transpose = X.T
    
    # 2. Calculate (X^T * X)
    XTX = np.dot(X_transpose, X)
    
    # 3. Calculate Inverse: (X^T * X)^-1
    XTX_inv = np.linalg.inv(XTX)
    
    # 4. Calculate X^T * y
    XTy = np.dot(X_transpose, y)
    
    # 5. theta = (XTX_inv) * (XTy)
    theta = np.dot(XTX_inv, XTy)
    
    return theta

In [33]:
X_train_28 = np.array(X_train_27)
y_train_28 = np.array(y_train_27)
X_test_28 = np.array(X_test_27)
y_test_28 = np.array(y_test_27)

theta_linear = calculate_theta(X_train_28, y_train_28)
print(theta_linear)
bias = 0.3707226311142412

y_train_pred_28 = np.dot(X_train_28, theta_linear) + bias
train_mse_28 = np.mean((y_train_28 - y_train_pred_28)**2)
print(f"Train MSE: {train_mse_28:.4f}")
y_test_pred_28 = np.dot(X_test_28, theta_linear) + bias
test_mse_28 = np.mean((y_test_28 - y_test_pred_28)**2)
print(f"Test MSE: {test_mse_28:.4f}")

[-0.14381853  0.23736977 -0.05098997  0.03049283]
Train MSE: 0.1475
Test MSE: 0.3383
